In [1]:
from slippi.parse import parse
from slippi.parse import ParseEvent
from slippi import Game

import pandas as pd
import os
import glob
from datetime import datetime

## get slp files

In [2]:
# Get all of the files in the dataset
path = "dataset/cynthia/"

# get sets
sets = glob.glob(os.path.join(path, '*'))

games = []
# get games in sets
for s in sets:
    games += (glob.glob(os.path.join(s, '*')))

print(f'{len(games)=}, {games[0]=}')

len(games)=6, games[0]='dataset/cynthia/cynthia_cpu_day2/Game_20240430T091645.slp'


## get inputs from an slp file

find the port numbers used in the game

In [3]:
# get the port indices used by the players
def getPorts(game):
    ports = []
    for i in range(0, len(game.start.players)):
        if game.start.players[i] is not None:
            ports.append(i)

    return ports


decide the names of the files we will write the inputs to (two per game -- one for each player)

In [4]:
# determine filenames of extracted input files
def decideFileNames(path):
    # get the names of our 2 files
    names = path.split('/')[-2].split('_')[0:2] # get the 2 player names from the directory name

    # datetime for the game
    gametime = path.split('_')[-1].split('.')[0]

    # name for file w/ p1 inputs
    p1_name = names[0] + '_' + names[1] + '_' + gametime
    # name for file w/ p2 inputs
    p2_name = names[1] + '_' + names[0] + '_' + gametime

    return p1_name, p2_name

In [5]:
def getCharacters(game):
    characters = []
    for player in game.start.players:
        if player is not None:
            characters.append(player.character)
    
    return characters

get the inputs for every frame of the match

In [6]:
START_FRAME = 64 # game starts on frame 64
COLS = ['joy_x', 'joy_y', 'cstick_x', 'cstick_y', 'z', 'r_dig', 'l_dig', 'a', 'b', 'x', 'y']
BUTTONS = ['Z', 'R', 'L', 'A', 'B', 'X', 'Y'] # names of the buttons in py-slippi

def getFrameInputs(player):
    # analog stick / c stick
    analog = [player.joystick.x, player.joystick.y, player.cstick.x, player.cstick.y]

    # buttons
    pressed_buttons = []
    # get the names of the buttons currently being pressed
    logical_pressed_names = map(lambda x: x.name, player.buttons.physical.pressed())

    for b in BUTTONS:

        if b in logical_pressed_names:
            pressed_buttons.append(1)
        else:
            pressed_buttons.append(0)

    return analog + pressed_buttons 

def getGameInputs(game):
    p1_inputs = pd.DataFrame(columns=COLS)
    p2_inputs = pd.DataFrame(columns=COLS)

    # get the controller port #s of the players
    p1_port, p2_port = getPorts(game)

    for i in range(START_FRAME, len(game.frames)):
        p1 = game.frames[i].ports[p1_port].leader.pre 
        p1_inputs.loc[len(p1_inputs.index)] = getFrameInputs(p1)

        p2 = game.frames[i].ports[p2_port].leader.pre
        p2_inputs.loc[len(p2_inputs.index)] = getFrameInputs(p2)

    return p1_inputs, p2_inputs


### get inputs from all files & write them in csv form

run getGameInputs on every replay file

In [7]:
OUTPUT_DIR = 'dataset/demo/inputs/'

for file in games:
    game = Game(file)
    p1_inputs, p2_inputs = getGameInputs(game)
    p1_file, p2_file = decideFileNames(file)

    # print some info for sanity checking
    print(f'{p1_file}, {p2_file}')
    print(getCharacters(game))    

    p1_inputs.to_csv(OUTPUT_DIR + p1_file, index=False)
    p2_inputs.to_csv(OUTPUT_DIR + p2_file, index=False)


cynthia_cpu_20240430T091645, cpu_cynthia_20240430T091645
[12:PEACH, 10:MEWTWO]
cynthia_cpu_20240430T092207, cpu_cynthia_20240430T092207
[12:PEACH, 14:ICE_CLIMBERS]
cynthia_cpu_20240430T091131, cpu_cynthia_20240430T091131
[12:PEACH, 23:ROY]
cynthia_cpu_20240429T163412, cpu_cynthia_20240429T163412
[12:PEACH, 8:MARIO]
cynthia_cpu_20240429T164626, cpu_cynthia_20240429T164626
[12:PEACH, 2:FOX]
cynthia_cpu_20240429T164109, cpu_cynthia_20240429T164109
[12:PEACH, 5:BOWSER]
